<a href="https://colab.research.google.com/github/grosa1/hands-on-ml-tutorials/blob/master/tutorial_1/pydriller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with PyDriller


PyDriller is a Python framework that helps developers on mining software repositories. With PyDriller you can easily extract information from any Git repository, such as commits, developers, modifications, diffs, and source codes, and quickly export CSV files.

## Installation and import

In [1]:
!pip install pydriller==1.14

We also import `pandas` to deal with data and `datetime` to work with dates:

In [0]:
from pydriller import RepositoryMining
import pandas as pd
from datetime import datetime

## Commit Object
A Commit object has all the information of a Git commit, and much more. More specifically:
```
hash (str): hash of the commit
msg (str): commit message
author (Developer): commit author (name, email)
author_date (datetime): authored date
author_timezone (int): author timezone (expressed in seconds from epoch)
committer (Developer): commit committer (name, email)
committer_date (datetime): commit date
committer_timezone (int): commit timezone (expressed in seconds from epoch)
branches (List[str]): List of branches that contain this commit
in_main_branch (Bool): True if the commit is in the main branch
merge (Bool): True if the commit is a merge commit
modifications (List[Modifications]): list of modified files in the commit (see Modifications)
parents (Set[str]): list of the commit parents
project_name (str): project name
project_path (str): project path
```

## Mining single project commits

In [0]:
project_url = 'https://github.com/ishepard/pydriller.git'

repo_commits = list()
for commit in RepositoryMining(project_url).traverse_commits():
    repo_commits.append({
        'sha': commit.hash, 
        'author': commit.author.name, 
        'date': commit.author_date
        })

In [4]:
pd.DataFrame(repo_commits)

,sha,author,date
0,ab36bf45859a210b0eae14e17683f31d19eea041,ishepard,2018-03-21 16:34:21+01:00
1,fdf671856b260aca058e6595a96a7a0fba05454b,ishepard,2018-03-22 11:07:31+01:00
2,90ca34ebfe69629cb7f186a1582fc38a73cc572e,ishepard,2018-03-22 12:53:52+01:00
3,71e053f61fc5d31b3e31eccd9c79df27c31279bf,ishepard,2018-03-26 13:13:27+02:00
4,205f6fb09734667b0c1842fd3c317013640189ce,ishepard,2018-03-27 16:34:02+02:00
...,...,...,...
556,b740a57d5969695270075fc818021f7c0b668bf1,Arie van Deursen,2020-04-22 18:15:05+02:00
557,a7b29f804238003b05113845e9f949c56647434a,Arie van Deursen,2020-04-23 12:48:07+02:00
558,3e7f8443636de673776446653988e6b1cb470ed9,Arie van Deursen,2020-04-26 22:46:04+02:00
559,955f9e736e7a97961b8ab6c76eea4d6c0d54c0e2,Davide Spadini,2020-04-27 10:00:08+02:00


## Mining multiple project commits

In [0]:
repos_url = [ "https://github.com/TheAlgorithms/Java.git", "https://github.com/apache/netbeans" ]

commits = list()
for commit in RepositoryMining(path_to_repo=repos_url).traverse_commits():
    commits.append({
        'sha': commit.hash, 
        'author': commit.author.name, 
        'msg': commit.msg
        })

In [6]:
pd.DataFrame(commits)

,sha,author,msg
0,40d42574e065e8078b242d201e0fc1455c430c71,Anup Kumar Panwar,Initial commit
1,4ba958863b0cd2212b681598969bf92450c13b71,Anup Kumar Panwar,Bubble Sort
2,12d7c48ee4b7f415f19dda4a889032263cc3529a,Anup Kumar Panwar,insertion sort
3,9661bb3df62929cad320ce576c7e156a7c91a748,Anup Kumar Panwar,Binary Search
4,73bb72b8d0ab1e3a8706394782c399d4e20f3308,Anup Kumar Panwar,Renamed
...,...,...,...
4871,12ec73da0b532f2e2f3b1f25f3210cee39a1db7f,Akshay-Gupta-Oracle,[NETBEANS-3311] Convert TextBlock to String Fu...
4872,98c980cbf3c519fbffc26380ede71e0ca4bb9eef,Eric Barboni,Merge pull request #2092 from jlahoda/NETBEANS...
4873,0dc75ef3899d27a9f8a917bfa79cd9e2abcf69c1,Eric Barboni,Merge pull request #2077 from svenreimers/issu...
4874,9e0096fa1fe6f11c3fb85b413db0cdb204d6c7a3,Akhilesh Singh,[NETBEANS-3985] : Addressed code review comments


## Get modifications
A modification object has the following fields:
```
old_path: old path of the file (can be None if the file is added)
new_path: new path of the file (can be None if the file is deleted)
change_type: type of the change: can be Added, Deleted, Modified, or Renamed.
diff: diff of the file as Git presents it (e.g., starting with @@ xx,xx @@).
source_code: source code of the file (can be None if the file is deleted)
source_code_before: source code of the file before the change (can be None if the file is added)
added: number of lines added
removed: number of lines removed
nloc: Lines Of Code (LOC) of the file
complexity: Cyclomatic Complexity of the file
token_count: Number of Tokens of the file
methods: list of methods of the file. The list might be empty if the programming language is not supported or if the file is not a source code file.
```

To get the list of `Modifications` that exists inside a `Commit` object:

In [0]:
mod_commits = list()
for commit in RepositoryMining('https://github.com/TheAlgorithms/Java.git').traverse_commits():
    for m in commit.modifications:
        mod_commits.append({
            'author': commit.author.name,
            'modified_file': m.filename,
            'change_type': m.change_type.name,
            'cyclomatic_complexity': m.complexity
            })

In [8]:
pd.DataFrame(mod_commits)

,author,modified_file,change_type,cyclomatic_complexity
0,Anup Kumar Panwar,README.md,ADD,0.0
1,Anup Kumar Panwar,BubbleSort.java,ADD,6.0
2,Anup Kumar Panwar,InsertionSort.java,ADD,6.0
3,Anup Kumar Panwar,SelectionSort.java,ADD,6.0
4,Anup Kumar Panwar,BinarySearch.java,ADD,7.0
...,...,...,...,...
1672,EAlexa,SinglyLinkedList.java,MODIFY,35.0
1673,EAlexa,MinimizingLateness.java,MODIFY,6.0
1674,markaster,LICENSE,MODIFY,0.0
1675,LittleFoot,ShellSort.java,MODIFY,7.0


## Filter by commit

In [9]:
url = "https://github.com/ishepard/pydriller.git"
for commit in RepositoryMining(url, single='05526fad873c3fc83e40bcbc424bd1b3e5393dd5').traverse_commits():
    print('Hash {}, author {}'.format(commit.hash, commit.author.name))

Hash 05526fad873c3fc83e40bcbc424bd1b3e5393dd5, author ishepard


## Filter by date

In [0]:
filtered_commits = list()
for commit in RepositoryMining(url, since=datetime(2020, 1, 1, 1, 0, 0)).traverse_commits():
    filtered_commits.append({
        'sha': commit.hash, 
        'author': commit.author.name, 
        'msg': commit.msg
        })

In [11]:
pd.DataFrame(filtered_commits)

,sha,author,msg
0,c69e50b5d68b42c19639ac81c37f039581e149ad,stefanodallapalma,Added metric to count for devs who contributed...
1,9baf4fd9e1cb84546ae9fe6864e158b3a1c01080,stefanodallapalma,Added process metric to count the number of ne...
2,22573d99f7135d37d0aab9cd8fee0ae9ec1b6c49,stefanodallapalma,Added two process metrics
3,9be1a6f6e420ae19303b6a94caccf93397068d02,stefanodallapalma,Added metric to count normalized number of add...
4,8e379834929c14f2da5fb2cba04e2326f6ef0a3a,stefanodallapalma,Added metric to count the normalized number of...
...,...,...,...
151,b740a57d5969695270075fc818021f7c0b668bf1,Arie van Deursen,Fix incorrect type annotation
152,a7b29f804238003b05113845e9f949c56647434a,Arie van Deursen,Rename TEST_DATA variable when re-used to reso...
153,3e7f8443636de673776446653988e6b1cb470ed9,Arie van Deursen,Make variable type annotations python 3.5 comp...
154,955f9e736e7a97961b8ab6c76eea4d6c0d54c0e2,Davide Spadini,Merge pull request #115 from avandeursen/mypy\...


## Other options
PyDriller comes with a set of common commit filters that you can apply:
```
only_in_branch (str): only analyses commits that belong to this branch.
only_no_merge (bool): only analyses commits that are not merge commits.
only_authors (List[str]): only analyses commits that are made by these authors. The check is made on the username, NOT the email.
only_commits (List[str]): only these commits will be analyzed.
only_releases (bool): only commits that are tagged (“release” is a term of GitHub, does not actually exist in Git)
filepath (str): only commits that modified this file will be analyzed.
only_modifications_with_file_types (List[str]): only analyses commits in which at least one modification was done in that file type, e.g., if you pass “.java”, it will visit only commits in which at least one Java file was modified; clearly, it will skip other commits (e.g., commits that did not modify Java files).
```

In [0]:
# Only commits in branch1 and no merges
RepositoryMining('path/to/the/repo', only_in_branch='branch1', only_no_merge=True).traverse_commits()

# Only commits of author "ishepard"
RepositoryMining('path/to/the/repo', only_authors=['ishepard']).traverse_commits()

# Only commits that modified a java file
RepositoryMining('path/to/the/repo', only_modifications_with_file_types=['.java']).traverse_commits()

## Resources

- PyDriller docs: [link](https://pydriller.readthedocs.io/) and [link](https://readthedocs.org/projects/pydriller/downloads/pdf/latest/)
- PyDriller source code: [link](https://github.com/ishepard/pydriller)
- PyDriller paper: [link](https://www.sback.it/publications/fse2018td.pdf)
- From: [link](https://www.kaggle.com/sayedmohsin/pydriller-tool-demo-by-sayed-mohsin-reza#Pydriller-Tool-Demo)